In [1]:
%cd /Users/wg/Proj/tickettoride
from ticket_to_ride import *

import numpy as np

/Users/wg/Proj/tickettoride


In [9]:
r = ticket_to_ride.routes[0]
r.from_city, r.to_city

('Vancouver', 'Calgary')

In [2]:
graph = dict()
for city in ticket_to_ride.cities:
    graph[city] = set()
    for route in ticket_to_ride.routes:
        if route.from_city == city:
            graph[city].add(route.to_city)
        elif route.to_city == city:
            graph[city].add(route.from_city)

In [3]:
graph

{'Atlanta': {'Charleston', 'Miami', 'Nashville', 'New Orleans'},
 'Boston': {'Montreal', 'New York'},
 'Calgary': {'Helena', 'Seattle', 'Vancouver', 'Winnepeg'},
 'Charleston': {'Atlanta', 'Miami', 'Raleigh'},
 'Chicago': {'Duluth', 'Omaha', 'Pittsburgh', 'Saint Louis', 'Toronto'},
 'Dallas': {'Houston', 'Little Rock', 'Oklahoma City'},
 'Denver': {'Helena',
  'Kansas City',
  'Oklahoma City',
  'Omaha',
  'Phoenix',
  'Salt Lake City'},
 'Duluth': {'Chicago',
  'Helena',
  'Omaha',
  'Sault St. Marie',
  'Toronto',
  'Winnepeg'},
 'El Paso': {'Los Angeles', 'Phoenix', 'Santa Fe'},
 'Helena': {'Calgary',
  'Denver',
  'Duluth',
  'Omaha',
  'Salt Lake City',
  'Seattle',
  'Winnepeg'},
 'Houston': {'Dallas', 'New Orleans'},
 'Kansas City': {'Denver', 'Omaha', 'Saint Louis'},
 'Las Vegas': {'Los Angeles', 'Salt Lake City'},
 'Little Rock': {'Dallas',
  'Nashville',
  'New Orleans',
  'Oklahoma City',
  'Saint Louis'},
 'Los Angeles': {'El Paso', 'Las Vegas', 'Phoenix', 'San Francisco'},

In [54]:
home = 'New York'

for tier_1_city in graph[home]:
    print home, '->', tier_1_city
    for tier_2_city in graph[tier_1_city]:
        if tier_1_city == home or tier_2_city == home: continue
        print home, '->', tier_1_city, '->', tier_2_city

New York -> Pittsburgh
New York -> Pittsburgh -> Toronto
New York -> Pittsburgh -> Chicago
New York -> Pittsburgh -> Nashville
New York -> Pittsburgh -> Washington
New York -> Pittsburgh -> Saint Louis
New York -> Pittsburgh -> Raleigh
New York -> Boston
New York -> Boston -> Montreal
New York -> Washington
New York -> Washington -> Pittsburgh
New York -> Washington -> Raleigh
New York -> Montreal
New York -> Montreal -> Toronto
New York -> Montreal -> Boston
New York -> Montreal -> Sault St. Marie


In [ ]:


class Djikstra(object):
    
    def __init__(self):
        self.edge_distances = EdgeDistances.weights
        
    def run(self, initial_city):
        self.total_distances = [float("inf") for _ in range(len(cities))]
        self.total_distances[initial_city] = 0
        self.current = initial_city
        self.unvisited = range(len(self.total_distances))
        self.visit_queue = []
        self.histories = [float("nan") for _ in range(len(cities))]

        while True:
            if not len(self.unvisited):
                break
            self.step()
            
    def visited_target(self, target):
        return target not in self.unvisited
        
    def step(self):
        self.assign_distances()
        self.reposition()
        
    def assign_distances(self):
        d_0 = self.total_distances[self.current]
        tentative_distances = self.distance_from(self.current)
        for i, d in enumerate(tentative_distances):
            if i not in self.unvisited:
                continue
            elif self.total_distances[i] > d + d_0:
                self.total_distances[i] = d + d_0
                self.histories[i] = self.current
                
    def distance_from(self, ix):
        return self.edge_distances[ix]
    
    def neighbors(self):
        enumerated = enumerate(self.edge_distances[self.current])
        return [i for i, x in enumerated if not math.isinf(x)]
    
    def reposition(self):
        self.unvisited.remove(self.current)
        self.visit_queue += self.neighbors()
        while self.current not in self.unvisited and len(self.visit_queue):
            self.current = self.visit_queue.pop(0)
            
    def show_best_route_to(self, target):
        k = []
        r = self.histories[target]
        while not math.isnan(r):
            k.append(r)
            r = self.histories[r]
        return list(reversed(k))+[target]

In [90]:
    def bfs(graph, start):
        visited, queue = set(), [start]
        depth = 0
        prev = {start: []}
        distances = {}
        while queue:
            vertex = queue.pop(0)
            prev[vertex]
            if vertex not in visited:
                visited.add(vertex)
                neighbors = graph[vertex] - visited
                queue.extend(neighbors)
                for neighbor in neighbors:
                    if len(prev[vertex]) + 1 < distances.get(neighbor, np.inf):
                        prev[neighbor] = prev[vertex] + [vertex]
                        distances[neighbor] = len(prev[vertex]) + 1
        return distances

In [ ]:
def reconstruct_path(cameFrom, current):
    total_path = set(current)
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.add(current)
    return total_path

def A_Star(start, goal):
    # The set of nodes already evaluated
    closedSet = set()

    # The set of currently discovered nodes that are not evaluated yet.
    # Initially, only the start node is known.
    openSet = set(start)

    # For each node, which node it can most efficiently be reached from.
    # If a node can be reached from many nodes, cameFrom will eventually contain the
    # most efficient previous step.
    cameFrom = dict()

    # For each node, the cost of getting from the start node to that node.
    gScore = defaultdict(1e6)

    # The cost of going from start to start is zero.
    gScore[start] = 0

    # For each node, the total cost of getting from the start node to the goal
    # by passing by that node. That value is partly known, partly heuristic.
    fScore = defaultdict(1e6)

    # For the first node, that value is completely heuristic.
    fScore[start] = heuristic_cost_estimate(start, goal)

    while len(openSet):
        # the node in openSet having the lowest fScore[] value
        current = filter(lambda node: fScore[node] == min(fScore.values()), openSet)[0] 
        if current == goal:
            return reconstruct_path(cameFrom, current)

        openSet.remove(current)
        closedSet.add(current)

        for neighbor in current:
            if neighbor in closedSet:
                continue		# Ignore the neighbor which is already evaluated.

            if neighbor not in openSet:	# Discover a new node
                openSet.Add(neighbor)
            
            # The distance from start to a neighbor.
            # The "dist_between" function may vary as per the solution requirements.
            tentative_gScore = gScore[current] + dist_between(current, neighbor)
            if tentative_gScore >= gScore[neighbor]:
                continue		# This is not a better path.

            # This path is the best until now. Record it!
            cameFrom[neighbor] = current
            gScore[neighbor] = tentative_gScore
            fScore[neighbor] = gScore[neighbor] + heuristic_cost_estimate(neighbor, goal)

In [109]:
class Graph:
    graph = dict()
    for city in ticket_to_ride.cities:
        graph[city] = set()
        for route in ticket_to_ride.routes:
            if route.from_city == city:
                graph[city].add(route.to_city)
            elif route.to_city == city:
                graph[city].add(route.from_city)

In [108]:
MaxDistance.find('New York', 'San Francisco')

6

In [7]:
t = Ticket('New York', 'Washington', 3)

from itertools import chain
def satisfied(start, routes):
    available = list(chain(*map(lambda r: [r.from_city, r.to_city], routes)))
    visited, queue = set(), [start]
    depth = 0
    prev = {start: []}
    distances = {}
    while queue:
        vertex = queue.pop(0)
        prev[vertex]
        if vertex not in visited and vertex in available:
            visited.add(vertex)
            neighbors = graph[vertex] - visited
            queue.extend(neighbors)
            for neighbor in neighbors:
                if len(prev[vertex]) + 1 < distances.get(neighbor, np.inf):
                    prev[neighbor] = prev[vertex] + [vertex]
                    distances[neighbor] = len(prev[vertex]) + 1
    return distances

In [30]:
def route_graph_from_routes(routes):
    graph = dict()
    for city in cities:
        graph[city] = set()
        for route in routes:
            if route.from_city == city:
                graph[city].add(route.to_city)
            elif route.to_city == city:
                graph[city].add(route.from_city)
    return graph

In [31]:
routes2 = [
		Route('Vancouver', 'Calgary', 3, '-'),
		Route('Vancouver', 'Seattle', 1, '-'),
		Route('Seattle', 'Portland', 1, '-'),
		Route('San Francisco', 'Los Angeles', 1, '-')
	]

{'Atlanta': set(),
 'Boston': set(),
 'Calgary': {'Vancouver'},
 'Charleston': set(),
 'Chicago': set(),
 'Dallas': set(),
 'Denver': set(),
 'Duluth': set(),
 'El Paso': set(),
 'Helena': set(),
 'Houston': set(),
 'Kansas City': set(),
 'Las Vegas': set(),
 'Little Rock': set(),
 'Los Angeles': {'San Francisco'},
 'Miami': set(),
 'Montreal': set(),
 'Nashville': set(),
 'New Orleans': set(),
 'New York': set(),
 'Oklahoma City': set(),
 'Omaha': set(),
 'Phoenix': set(),
 'Pittsburgh': set(),
 'Portland': {'Seattle'},
 'Raleigh': set(),
 'Saint Louis': set(),
 'Salt Lake City': set(),
 'San Francisco': {'Los Angeles'},
 'Santa Fe': set(),
 'Sault St. Marie': set(),
 'Seattle': {'Portland', 'Vancouver'},
 'Toronto': set(),
 'Vancouver': {'Calgary', 'Seattle'},
 'Washington': set(),
 'Winnepeg': set()}